<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       BincodeFit and BincodeTransform functions in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Bin Coding, also known as binning or bucketing, is a data preprocessing technique used in statistics and machine learning to transform continuous data into categorical or discrete data. In binning, a range of continuous values is divided into smaller intervals or bins, and the data values are assigned to the appropriate bin. This allows the data to be analyzed more easily by grouping similar values into categories.<br> In this notebook we will see how we can use the BincodeFit and BincodeTransform function available in Vantage.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import *

# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_BincodeFitandTransform_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_cloud');"        # Takes 30 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_BankChurn_local');" 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
tdf = DataFrame(in_schema("DEMO_BankChurn", "customer_churn"))
print("Shape of the data: ", tdf.shape)
tdf

In [ ]:
tdf.tdtypes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Detailed help can be found by passing function name to built-in help function. </p>

``` python 
help(BincodeFit)
help(BincodeTransform)
```

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create bins for the Age column.<br>For option 1 we will create bin table and pass that to the BincodeFit function.<br>Bins can also be created using Histogram function.</p>

In [ ]:
%%capture
query1 = '''
CREATE MULTISET TABLE DEMO_User.FitInputTable 
     (
      ColumnName varchar(20), 
      MinValue integer, 
      MaxValue integer, 
      Label varchar(20)
  )
NO PRIMARY INDEX;
'''

query2 = '''
insert into FitInputTable values('age', 0, 18, '1-Children')
;insert into FitInputTable values('age', 19, 25, '2-Young Adult')    
;insert into FitInputTable values('age', 26, 45, '3-Middle Adult')
;insert into FitInputTable values('age', 46, 60, '4-Old Adult')   
;insert into FitInputTable values('age', 61 ,120, '5-Senior Citizen')
;
'''

execute_sql(query1)
execute_sql(query2)

In [ ]:
bin_data=DataFrame(in_schema("DEMO_User","FitInputTable"))
bin_data

In [ ]:
bin_fit_1 = BincodeFit(data=tdf,
                        fit_data=bin_data,
                        fit_data_order_column = ['MinValue', 'MaxValue'],
                        target_columns='Age',
                        minvalue_column='MinValue',
                        maxvalue_column='MaxValue',
                        label_column='Label',
                        method_type='Variable-Width'
                        )
 
# Print the result.
bin_fit_1.output

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now we can apply the fit function or transform the data using BincodeTransform</p>

In [ ]:
column_list = tdf.columns
column_list.remove("Age")

#res = fit_object.transform(data = tdf).result
res = BincodeTransform(data=tdf,
                        object=bin_fit_1,
                        object_order_column="TD_MinValue_BINFIT",
                        accumulate=column_list
                       )
res.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For option 2 Fit table itself creates and equal-width bins.</p>

In [ ]:
bin_fit_2 = BincodeFit(data=tdf,
                        target_columns='Age',
                        method_type='Equal-Width',
                        nbins=5,
                        label_prefix='Age_Category'
                        )
bin_fit_2.output

In [ ]:
res_2 = BincodeTransform(data=tdf,
                        object=bin_fit_2,
                        accumulate=column_list
                       )
res_2.result

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Cleanup</b>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
db_drop_table(table_name = 'FitInputTable')

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_BankChurn');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Dataset:</b>

- `Surname`: Surname
- `CreditScore`: Credit score
- `Geography`: Country (Germany / France / Spain)
- `Gender`: Gender (Female / Male)
- `Age`: Age
- `Tenure`: No of years the customer has been associated with the bank
- `Balance`: Balance
- `NumOfProducts`: No of bank products used
- `HasCrCard`: Credit card status (0 = No, 1 = Yes)
- `IsActiveMember`: Active membership status (0 = No, 1 = Yes)
- `EstimatedSalary`: Estimated salary
- `Exited`: Abandoned or not? (0 = No, 1 = Yes)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>BincodeFit function reference: <a href = 'https://docs.teradata.com/search/all?query=BincodeFit&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>